In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [22]:

train_D = pd.read_csv("Davis_train_origin.csv", header = 0)
test_D = pd.read_csv("Davis_test_origin.csv", header = 0)
val_D = pd.read_csv("Davis_val_origin.csv", header = 0)

train_D.drop("Unnamed: 0", axis = 1, inplace = True)
test_D.drop("Unnamed: 0", axis = 1, inplace = True)
val_D.drop("Unnamed: 0", axis = 1, inplace = True)

unique_drug = test_D['Drug_ID'].value_counts().index + test_D['Drug_ID'].value_counts().index + val_D['Drug_ID'].value_counts().index

for i in unique_drug :
    print(i)
    
print(len(unique_drug))

24779724
11626560
16007391
176870
5329102
16725726
9926054
3081361
123631
11617559
11712649
126565
11485656
11234052
216239
6918454
5287969
153999
5328940
9884685
44259
51004351
16722836
5494449
3025986
9809715
42642645
6450551
11984591
176167
10138260
5291
11667893
156414
11364421
10461815
11314340
16038120
11338033
9926791
9915743
25127112
644241
3062316
24889392
11717001
3038522
176155
3038525
9929127
10127622
10113978
44150621
24180719
4908365
208908
11409972
25243800
17755052
10184653
10427712
156422
151194
11427553
9933475
447077
11656518
10074640
68


In [79]:
protein2gene = pd.read_csv("convert_table", delimiter="\t")

In [187]:
pubchem2drugbank =  pd.read_csv("mbrole2_conversion06232318.tsv", delimiter='\t')

origin_dti =  val_D[val_D['Drug_ID'].isin(np.array(pubchem2drugbank['Input']))][['Drug_ID', "Target_ID", "Y"]]

def pub_to_db(x) : 
    return pubchem2drugbank[pubchem2drugbank['Input'] == x]['Output'].values[0]

def pro_to_gene(x) :
    
    result = protein2gene[protein2gene['Gene name'] == x]['NCBI gene (formerly Entrezgene) ID'].values
    
    if len(result) >= 1 :
        return result[0]
    else :
        return 1

dti = pd.DataFrame()

dti['Drug'] = origin_dti['Drug_ID'].apply(pub_to_db)
dti['Target'] = origin_dti['Target_ID'].apply(pro_to_gene)

In [188]:
dti['rel'] = origin_dti["Y"]

In [189]:
dti = dti[dti['Target'] != 1]

In [190]:
dti

,Drug,Target,rel
2,DB06589,Gene::8737,0
4,DB08142,Gene::9212,0
6,DB00619,Gene::3932,0
7,DB01259,Gene::4214,0
11,DB02567,Gene::8569,0
...,...,...,...
2567,DB04879,Gene::2044,0
2569,DB01259,Gene::207,0
2570,DB00619,Gene::4485,0
2571,DB06616,Gene::4751,0


In [191]:
import pandas as pd
import numpy as np
import json

with open('entity_drug.json', 'r') as f:
    data = json.load(f)

df = pd.json_normalize(data)

df = df.T

In [192]:
dti

,Drug,Target,rel
2,DB06589,Gene::8737,0
4,DB08142,Gene::9212,0
6,DB00619,Gene::3932,0
7,DB01259,Gene::4214,0
11,DB02567,Gene::8569,0
...,...,...,...
2567,DB04879,Gene::2044,0
2569,DB01259,Gene::207,0
2570,DB00619,Gene::4485,0
2571,DB06616,Gene::4751,0


In [193]:
#a=a.drop("gene_inco", axis = 1)
temp_list = []
for i in dti['Target'] :
    exist = False
    for cn, j in enumerate(df.index) :
        length = len(df.index)
        if i == j :
            exist = True
            val = int(df[0].loc[j])
            temp_list.append(val)
            break
        elif cn == length -1 :
            temp_list.append(np.nan)
            

In [194]:
dti['gene_inco'] = temp_list

In [195]:
dti

,Drug,Target,rel,gene_inco
2,DB06589,Gene::8737,0,3179.0
4,DB08142,Gene::9212,0,3113.0
6,DB00619,Gene::3932,0,2439.0
7,DB01259,Gene::4214,0,2032.0
11,DB02567,Gene::8569,0,3913.0
...,...,...,...,...
2567,DB04879,Gene::2044,0,7136.0
2569,DB01259,Gene::207,0,2232.0
2570,DB00619,Gene::4485,0,13250.0
2571,DB06616,Gene::4751,0,5991.0


In [196]:
import json

# JSON 파일 경로 설정
json_file = 'node2id.json'

# JSON 파일 읽기
with open(json_file) as f:
    data = json.load(f)
    
    
#data   

In [197]:
temp_list2 = [] 
for i in dti["Drug"].copy() :
    exist = False
    count = 0
    for name, number in data.items() :
        length = len(data)
        if i == name :
            exist = True
            temp_list2.append(number)
            break
        elif count >= length -1 :
            temp_list2.append(np.nan)
        count += 1
len(temp_list2)

670

In [199]:
dti[['gene_inco', 'drug_inco', 'rel']].to_csv('dev.txt', sep=' ', index=False)

In [198]:
dti['drug_inco'] = temp_list2
dti = dti.dropna()
dti['gene_inco'] = dti['gene_inco'].astype(int)
dti['drug_inco'] = dti['drug_inco'].astype(int)


/tmp/ipykernel_1036028/2201740663.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dti['gene_inco'] = dti['gene_inco'].astype(int)
/tmp/ipykernel_1036028/2201740663.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dti['drug_inco'] = dti['drug_inco'].astype(int)


In [185]:
dti

,Drug,Target,rel,gene_inco,drug_inco
20,DB05294,Gene::2051,0,6340,1185
24,DB01268,Gene::11183,0,8065,1018
26,DB00317,Gene::9874,0,10554,834
32,DB01268,Gene::8711,0,5836,1018
34,DB06589,Gene::2066,0,4034,437
...,...,...,...,...,...
5117,DB01254,Gene::4486,0,10145,764
5123,DB00530,Gene::1195,0,5967,600
5133,DB00619,Gene::23396,0,8909,637
5142,DB01259,Gene::494551,0,18827,206


In [113]:
t_d_g3['drug_inco'] = t_d_g3['drug_inco'].astype(int)
t_d_g3[["gene_inco", "drug_inco", "rel"]].to_csv("dti_dataset.csv", index=False)

/tmp/ipykernel_1045693/3045769039.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_d_g3['drug_inco'] = t_d_g3['drug_inco'].astype(int)


In [115]:
t_d_g3[["gene_inco", "drug_inco", "rel"]]

,gene_inco,drug_inco,rel
1,8627,1030,1
2,3636,1030,1
3,12417,668,1
4,11743,668,1
5,12128,804,1
...,...,...,...
9874,8382,1171,1
9875,13028,514,1
9876,8511,172,1
9877,15094,851,1


In [118]:
import pandas as pd
import numpy as np

# Load the dataset as a pandas DataFrame
df = t_d_g3[["gene_inco", "drug_inco", "rel"]]
# Shuffle the DataFrame rows
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split the dataset into training, testing, and validation sets
train_size = 0.7
test_size = 0.2
val_size = 0.1

train_split_index = int(len(df) * train_size)
test_split_index = int(len(df) * (train_size + test_size))

train_data = df.iloc[:train_split_index]
test_data = df.iloc[train_split_index:test_split_index]
val_data = df.iloc[test_split_index:]

# Print the shapes of the resulting datasets
print('Training data shape:', train_data.shape)
print('Testing data shape:', test_data.shape)
print('Validation data shape:', val_data.shape)

Training data shape: (6223, 3)
Testing data shape: (1777, 3)
Validation data shape: (890, 3)


In [120]:
test_data

,gene_inco,drug_inco,rel
6223,4252,260,1
6224,2987,1243,1
6225,5324,513,1
6226,1720,649,1
6227,18805,336,1
...,...,...,...
7995,4590,511,1
7996,7309,317,1
7997,12417,1097,1
7998,18786,203,1


hello


In [122]:
## negative sample generation

t_d_g3[["gene_inco", "drug_inco", "rel"]]



,gene_inco,drug_inco,rel
1,8627,1030,1
2,3636,1030,1
3,12417,668,1
4,11743,668,1
5,12128,804,1
...,...,...,...
9874,8382,1171,1
9875,13028,514,1
9876,8511,172,1
9877,15094,851,1


In [123]:
def make_set_by_values(list1, list2):
    # 두 리스트의 길이가 다르면 함수를 종료합니다.
    if len(list1) != len(list2):
        return None
    
    # set을 만들기 위한 빈 set 객체를 생성합니다.
    value_set = set()
    
    # zip 함수를 이용하여 두 리스트의 요소를 묶습니다.
    for val1, val2 in zip(list1, list2):
        # 묶은 요소를 이용하여 set에 새로운 요소를 추가합니다.
        value_set.add((val1, val2))
    
    # 생성한 set을 반환합니다.
    return value_set

In [127]:
make_set_by_values(t_d_g3["gene_inco"], t_d_g3["drug_inco"])

{(8339, 1208),
 (7415, 823),
 (16269, 1345),
 (6988, 490),
 (4252, 1015),
 (1991, 638),
 (4590, 1047),
 (3327, 1196),
 (8342, 1627),
 (7086, 540),
 (18777, 1306),
 (12128, 1037),
 (18835, 264),
 (9970, 1310),
 (14278, 159),
 (13261, 1532),
 (18803, 1480),
 (4563, 540),
 (5077, 761),
 (4590, 1168),
 (8339, 282),
 (3272, 398),
 (10536, 940),
 (5324, 1392),
 (12846, 243),
 (12115, 438),
 (18777, 380),
 (9274, 681),
 (7555, 587),
 (7601, 703),
 (15658, 615),
 (15658, 386),
 (5164, 1295),
 (18803, 554),
 (4054, 262),
 (10929, 992),
 (4590, 242),
 (7109, 914),
 (9110, 694),
 (12128, 1050),
 (10964, 949),
 (18777, 501),
 (18777, 272),
 (7128, 375),
 (8339, 524),
 (2914, 932),
 (12846, 485),
 (8682, 634),
 (12944, 562),
 (12128, 1171),
 (7869, 494),
 (10964, 1070),
 (18777, 393),
 (7128, 496),
 (6464, 1270),
 (7128, 267),
 (12128, 124),
 (8339, 645),
 (2766, 889),
 (4590, 484),
 (8511, 119),
 (4590, 255),
 (8607, 581),
 (8607, 352),
 (17738, 1640),
 (9097, 179),
 (18777, 514),
 (7801, 188),
 (

In [128]:
import random

rel_true = make_set_by_values(t_d_g3["gene_inco"], t_d_g3["drug_inco"])

gene = random.choice(t_d_g3["gene_inco"])
drug = random.choice(t_d_g3["drug_inco"])

while len(neg_list) > 8890 :
    gene = random.choice(t_d_g3["gene_inco"])
    drug = random.choice(t_d_g3["drug_inco"])
    
    

NameError: name 'my_list' is not defined

In [141]:
gene = random.choice(t_d_g3["gene_inco"])
drug = random.choice(t_d_g3["drug_inco"])

rel_true = make_set_by_values(t_d_g3["gene_inco"], t_d_g3["drug_inco"])

(gene, drug) in rel_true

False

In [170]:
import numpy as np

neg_gene = []
neg_drug = []
count = 0
while (True) :
    gene = np.random.choice(t_d_g3["gene_inco"])
    drug = np.random.choice(t_d_g3["drug_inco"])
    
    if (gene, drug) not in rel_true :
        neg_gene.append(gene)
        neg_drug.append(drug)
        count +=1
    if count >= 8890 :
        break

In [173]:
neg_df = pd.DataFrame({"gene_inco" : neg_gene, "drug_inco" : neg_drug})
neg_df["rel"] = 0

In [174]:
neg_df

,gene_inco,drug_inco,rel
0,13897,215,0
1,7801,34,0
2,5079,491,0
3,12944,263,0
4,3272,871,0
...,...,...,...
8885,5535,162,0
8886,18781,372,0
8887,14697,894,0
8888,5535,482,0


In [176]:
df_both = pd.concat([df, neg_df]).reset_index().drop(['index'], axis = 1)

In [180]:
import pandas as pd
import numpy as np

# Load the dataset as a pandas DataFrame

# Shuffle the DataFrame rows
df_both = df_both.sample(frac=1, random_state=42).reset_index(drop=True)

# Split the dataset into training, testing, and validation sets
train_size = 0.7
test_size = 0.2
val_size = 0.1

train_split_index = int(len(df_both) * train_size)
test_split_index = int(len(df_both) * (train_size + test_size))

train_data = df_both.iloc[:train_split_index]
test_data = df_both.iloc[train_split_index:test_split_index]
val_data = df_both.iloc[test_split_index:]

# Print the shapes of the resulting datasets
print('Training data shape:', train_data.shape)
print('Testing data shape:', test_data.shape)
print('Validation data shape:', val_data.shape)

train_data.to_csv('train.txt', index=False, sep=' ', header = False)
test_data.to_csv('test.txt', index=False, sep=' ', header = False)
val_data.to_csv('dev.txt', index=False, sep=' ', header = False)

Training data shape: (12446, 3)
Testing data shape: (3555, 3)
Validation data shape: (1779, 3)
